In [10]:
#导入必要的函数库
import numpy
import random
import pandas as pd
import math
import csv
from collections import defaultdict

In [11]:
#读取测试集数据
filename = "train.csv"
f = open(filename, "rt", encoding="utf-8")
header = f.readline()
header = header.strip().split(',')#列名
print(header)

['user_id', 'business_id', 'date', 'stars']


In [12]:
dataset = []
#将测试集数据放入dataset
for line in f:
    fields = line.strip().split(',')
    d = dict(zip(header, fields))
    d["stars"] = float(d["stars"])
    dataset.append(d)

In [21]:
ItemPerUser = defaultdict(set)
for d in dataset:
    user,item = d['user_id'], d['business_id']
    ItemPerUser[user].add(item)

In [22]:
CandidateItems = defaultdict(list)
for d in dataset:
    item = d['business_id']
    ItemPerUser[user].add(item)

In [28]:
ItemPerUser['A2JGzkvNjckSmps_4FbKWw']

{'FI-8LPGkc8D_hwx0l0zyyg',
 'OGRB__fguKfGpPdH7FvBDA',
 'PRbIMseOwPCCRv5C4SfTDQ',
 'Xg5qEQiB-7L6kGJ5F4K3bQ',
 'mPfdYJj4d5KBPINDkaZY-Q',
 'tWiFat101ID5w_wgAPMXhA',
 'umrDQGRNied77aVg29_fVw'}

In [23]:
def RandomSelectNegativeSample(items):
    ret = dict()
    for i in items.keys():
        ret[i] = 1
    n=0
    for i in range(0, len(items) * 3):
        item = items_pool[random.randint(0, len(items_pool) - 1)]
        if item in ret:
             continue
        ret[item] = 0
        n+=1
        if n > len(items):
             break
    return ret

In [24]:
RandomSelectNegativeSample(CandidateItems)

{}

In [9]:
def LatentFactorModel(user_items, N = 20, F = 100, alpha = 0.02, lambdaa = 0.01):
    [P, Q] = InitModel(user_items, F)
    for step in range(0,N):
        for user, items in user_items.items():
            samples = RandSelectNegativeSamples(items)
            for item, rui in samples.items():
                eui = rui - Predict(user, item)
                for f in range(0, F):
                    P[user][f] += alpha * (eui * Q[item][f] - lambdaa * P[user][f])
                    Q[item][f] += alpha * (eui * P[user][f] - lambdaa * Q[item][f])
        alpha *= 0.9
        
def Recommend(user, P, Q):
    rank = dict()
    for f, puf in P[user].items():
        for i, qfi in Q[f].items():
            if i not in rank:
                rank[i] += puf * qfi
    return rank
                              